In [1]:
import pandas as pd
import numpy as np

In [2]:
# code for reading all HTLM files in the directory

In [3]:
import os, sys

wp_path = "/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/" # to adapt
directories = os.listdir(wp_path)

html_list = []
for dir in directories:
    local_path = wp_path + dir
    local_dir = os.listdir(local_path)
    for link in local_dir:
        if link[-3:]=='htm':
            html_list.append(dir+'/'+link)
        
html_list=html_list[:100]
html_list

['r/Royal_Marines.htm',
 'r/Recycling.htm',
 'r/Retreat_of_glaciers_since_1850.htm',
 'r/Remember_Girls_Like_Rajpura.htm',
 'r/Rutherfordium.htm',
 'r/Rule_of_St_Benedict.htm',
 'r/Red_dwarf.htm',
 'r/Romeo_and_Juliet.htm',
 'r/Rudyard_Kipling.htm',
 'r/Race.htm',
 'r/Ruthenium.htm',
 'r/Rail_transport_in_India.htm',
 'r/Renormalization.htm',
 'r/Rosetta_Stone.htm',
 'r/Rupee.htm',
 'r/Rondane_National_Park.htm',
 'r/Russian_constitutional_crisis_of_1993.htm',
 'r/Richard_Wagner.htm',
 'r/Roman_mythology.htm',
 'r/Rudolf_Vrba.htm',
 'r/Richard_Nixon.htm',
 'r/Roman_Catholic_Church.htm',
 'r/Richard_Feynman.htm',
 'r/Roman_law.htm',
 'r/Royal_Grammar_School_Worcester.htm',
 'r/Rosemary.htm',
 'r/Roman_road.htm',
 'r/Royal_Geographical_Society.htm',
 'r/Radio_telescope.htm',
 'r/Rome.htm',
 'r/Royal_Dutch_Shell.htm',
 'r/Refrigerator_car.htm',
 'r/Roald_Amundsen.htm',
 'r/Roman_villa.htm',
 'r/Robert_Oppenheimer.htm',
 'r/Rabat.htm',
 'r/Richard_Stallman.htm',
 'r/Reigate_Wow_Balloon_Lau

The aim of this specific game strategy analysis is to test whether Wikispeedia players have clikability preferences on the positioning of the hyperlinks in the article text. Firstly we will address their postioning in the text, evaluated on the basis of the ordinal number of the paragraph they show up in; secondly we will test if there is a correlation between the clickthrough rate of the hyperlinks and their location in image captions. 

It is therefore essential to extract from the data the clicktrhough rate for each hyperlink.

In [4]:
# code for extracting the hyperlinks click frequency  

In [5]:
folder='/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wikispeedia_paths-and-graph/'

paths_finished='paths_finished.tsv'
articles='articles.tsv'

paths_finished=pd.read_csv(folder+paths_finished, sep='\t', skiprows=15, header=None, names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating'])
articles=pd.read_csv(folder+articles, sep='\t', skiprows=12, header=None, names=['article'])

This dataframe contains the paths chosen by the players who could reach the target article.

In [6]:
paths_finished.head()

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0


This dataframe lists all the clickable articles.

In [25]:
def filter_article_name(article_list):
    for article in article_list:
        print(article)
        article = article.lower()
        article = article.replace('_', ' ')
        article = article.strip()
        article_list
    return article_list

#articles.article=filter_article_name(articles.article)
#articles.head(100)

filter_article_name(['banana', 'CIAO', 'C_bb'])

banana
banana
CIAO
ciao
C_bb
c bb


['banana', 'CIAO', 'C_bb']

In [17]:
# this is the articles list in the lists of paths
    
clicked_articles=[]
for x in paths_finished['path']: # hypothesize also in paths_unfinished
    path=(x.split(';'))
    clicked_articles.append(path)

clicked_articles_filtered=[]
for path in clicked_articles:
    print(path)
    print('lll')
    path_filtered=filter_article_name(path)
    print(path_filtered)
    print('ppp')
    clicked_articles_filtered.append(path_filtered)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ppp
['Theatre', 'France', 'Sub-Saharan_Africa', 'Central_African_Republic', 'Lion', 'Zebra']
lll
['Theatre', 'France', 'Sub-Saharan_Africa', 'Central_African_Republic', 'Lion', 'Zebra']
ppp
['Theatre', 'Lighting_designer', '<', 'Russia', 'Earth', 'Animal', 'Mammal', 'Zebra']
lll
['Theatre', 'Lighting_designer', '<', 'Russia', 'Earth', 'Animal', 'Mammal', 'Zebra']
ppp
['Theatre', 'France', 'Kenya', 'Lion', 'Zebra']
lll
['Theatre', 'France', 'Kenya', 'Lion', 'Zebra']
ppp
['Theatre', 'India', 'Mammal', 'Zebra']
lll
['Theatre', 'India', 'Mammal', 'Zebra']
ppp
['Theatre', 'Brazil', 'Africa', 'Lion', 'Zebra']
lll
['Theatre', 'Brazil', 'Africa', 'Lion', 'Zebra']
ppp
['Theatre', 'Drama', 'Television', 'Film', 'Silent_film', '<', 'Photography', 'Video', 'Australia', 'Continent', 'Africa', 'Giraffe', 'Okapi', 'Zebra']
lll
['Theatre', 'Drama', 'Television', 'Film', 'Silent_film', '<', 'Photography', 'Video', 'Australia', 'Continent', 'Africa', 'Giraffe', 'Okapi', 'Zebra']
ppp
['Theatre', 'Dance',

In [10]:
# this is the dataframe containing the hyperlinks and their clicktrhough frequency

articles=articles.loc[:300]
clicked_articles_filtered=clicked_articles_filtered[:300]

links_name=[]
links_freq=[]

i=0

for article in articles['article']:
    count=0
    links_name.append(article)
    for i in range(len(clicked_articles_filtered)):
        if article in list(clicked_articles_filtered[i]):
            print(clicked_articles_filtered[i])
            count+=1 
        links_freq.append(count)

links_name=pd.Series(links_name)
links_freq=pd.Series(links_freq)
links_freq_df=pd.concat([links_name, links_freq], axis=1, names=['link', '#_clicks'])
links_freq_df.columns=['link', '#_clicks']
links_freq_df # don't know why it is not working

0               14th century
1                    england
2                     london
3             spanish armada
4                    england
5              great britain
6                          <
7               10th century
8               11th century
9               12th century
10              13th century
11              14th century
12              15th century
13              16th century
14             francis drake
15    elizabeth i of england
dtype: object
0               14th century
1                    england
2                     london
3             spanish armada
4                    england
5              great britain
6                          <
7               10th century
8               11th century
9               12th century
10              13th century
11              14th century
12              15th century
13              16th century
14             francis drake
15    elizabeth i of england
dtype: object
0               14th century
1              

0                 achilles
1                   apollo
2                 saturn v
3            united states
4     christopher columbus
5                        <
6                  chicago
7    university of chicago
8                plutonium
9                americium
dtype: object
0            africa
1              gold
2             xenon
3    periodic table
4         americium
dtype: object
0            africa
1           diamond
2            carbon
3    periodic table
4         americium
dtype: object
0                    14th century
1                          europe
2                               <
3                         england
4                               <
5                    christianity
6                     anglicanism
7                   united states
8                    walt whitman
9                               <
10    poetry of the united states
11                              <
12                              <
13                              <
14          

,link,#_clicks
0,%c3%81ed%c3%a1n mac gabr%c3%a1in,0
1,%c3%85land,0
2,%c3%89douard manet,0
3,%c3%89ire,0
4,%c3%93engus i of the picts,0
...,...,...
95,able archer 83,0
96,abraham goldfaden,0
97,abraham lincoln,0
98,absinthe,0


In [ ]:
articles=articles.loc[:300]
clicked_articles_filtered=clicked_articles_filtered[:300]

if 'africa' in list(clicked_articles_filtered[0].loc[:]):
    print('yes')
clicked_articles_filtered[0].loc[:]

# need to understand why not working

In [ ]:
# code for extracting the positioning of the hyperlinks in the article text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
URL = '/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/a/A_cappella.htm'

f=open(URL, 'r')
soup = BeautifulSoup(f, 'lxml')

In [ ]:
# read all the files

soups=[]
failed_html_list = []
for html in html_list[:]:
    f=open(wp_path+html, 'r')
    try:
        soup = BeautifulSoup(f, 'lxml')
        soups.append(soup)
    except Exception:
        failed_html_list.append(html)

In [ ]:
failed_html_list[:]

In [ ]:
# HTML tag for hyperlinks is <a href= >
# this is the dataframe of the hyperlinks contained in an article, with the number of paragraph they show up in 
# and its relative postion in the article

paragraphs = soup.find_all('p')

links=[]
i=0

for p in paragraphs:
    i+=1
    all_links=p.find_all('a')
    for link in all_links:
        if 'href' in link.attrs:
            link_title=link.text
            link_paragraph=i
            links.append([link_title, link_paragraph])

link_positioning_df=pd.DataFrame(links, columns=['link', '#_paragraph']).drop_duplicates()
link_positioning_df['positioning']=round(link_positioning_df['#_paragraph']/len(paragraphs), 2)
link_positioning_df['link']=filter_article_name(link_positioning_df['link'])
link_positioning_df

In [ ]:
# function for all html files

def find_links(soups):
    dfs=[]
    for soup in soups:
        paragraphs = soup.find_all('p')
        
        links=[]
        i=0
        for p in paragraphs:
            i+=1
            all_links=p.find_all('a')
            for link in all_links:
                if 'href' in link.attrs:
                    link_title=link.text
                    link_paragraph=i
                    links.append([link_title, link_paragraph])

        link_positioning_df=pd.DataFrame(links, columns=['link', '#_paragraph']).drop_duplicates()
        link_positioning_df['positioning']=round(link_positioning_df['#_paragraph']/len(paragraphs), 2)
        dfs.append(link_positioning_df)
    return dfs

link_positioning_dfs=find_links(soups)

In [ ]:
# code for finding hyperlinks in images' caption

In [ ]:
URL = '/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/a/Aachen.htm'
f2=open(URL, 'r')
soup2 = BeautifulSoup(f2, 'lxml')

In [ ]:
# in an HTML file the images can be either find under tables or div tags 

In [ ]:
# images of hyperlinks in tables' captions

tables = soup2.find_all('table')

links=[]
for table in tables:
    if table.find('img'):
        all_links=table.find_all('a')
        for link in all_links:
            if 'class' not in link.attrs:
                link_title=link.text
                links.append(link_title)

links_in_table_df=pd.DataFrame(links)
links_in_table_df.columns=['link_in_table']
links_in_table_df.apply(filter_article_name) 
links_in_table_df      

In [ ]:
# function for all html files

def find_links_in_table(soups):
    dfs=[]
    for soup in soups:
        tables = soup2.find_all('table')
        
        links=[]
        for table in tables:
            if d.find('img'):
            all_links=table.find_all('a')
            for link in all_links:
                if 'class' not in link.attrs:
                link_title=link.text
                links.append(link_title)

        links_in_table_df=pd.DataFrame(links).drop_duplicates() 
        links_in_table_df.columns=['link_in_table']
        dfs.append(links_in_table_df)
    return dfs

links_in_table_dfs=find_links_in_table(soups)

In [ ]:
# images under div tags

div = soup2.find_all('div')

links=[]

for d in div:
    if (d.find('a', class_="internal")):
        if (d.find('img', class_='thumbimage')):
            caption=d.find('div', class_="thumbcaption")
            if (caption.find('a')):
                all_links=caption.find_all('a')
                for link in all_links:
                    if 'class' not in link.attrs:
                        link_title=link.text
                        links.append(link_title)
                            
links_in_image_df=pd.DataFrame(links).drop_duplicates() 
links_in_image_df.columns=['link_in_image']
links_in_image_df.apply(filter_article_name)   

In [ ]:
# function for all html files

def find_links_in_div(soups):
    dfs=[]
    for soup in soups:
        div = soup2.find_all('div')
        links=[]
        for d in div:
            if (d.find('a', class_="internal")):
                if (d.find('img', class_='thumbimage')):
                    caption=d.find('div', class_="thumbcaption")
                    if (caption.find('a')):
                        all_links=caption.find_all('a')
                        for link in all_links:
                            if 'class' not in link.attrs:
                                link_title=link.text
                                links.append(link_title)

        links_in_image_df=pd.DataFrame(links).drop_duplicates() 
        links_in_image_df.columns=['link_in_image']
    return dfs

links_in_image_dfs=find_links_in_div(soups)

In [ ]:
links_name=[]
links_freq=[]

i=0
for article in articles['article']:
    count=0
    links_name.append(article)
    for i in range(len(clicked_articles_filtered)):
        if article in clicked_articles_filtered[i]:
            count+=1 
        links_freq.append(count)

links_name=pd.Series(links_name)
links_freq=pd.Series(links_freq)
links_freq_df=pd.concat([links_name, links_freq], axis=1, names=['link', '#_clicks'])
links_freq_df.columns=['link', '#_clicks']
links_freq_df.head(20) # don't know why it is not working

for df in links_in_image_dfs:
    

In [ ]:
# when we'll have all the articles' dataframes we will merge them by link and get the average of their positioning
# we will sort the articles based on their clickthrough rate, ascending=False
# we will check if there is a correlation between the positioning and clickthrough rate

In [ ]:
# when we'll have all the articles' dataframes we will label with a flag the hyperlinks located in images' captions
# we will check if there is a correlation between the location in an image caption and clickthrough rate

In [ ]:
# assumptions: 
# we don't know for the hyperlinks that show up both in the article text and in the image caption, 
# which the player actually clicked 
# for the hyperlinks which show up multiple times in the article ...


In [ ]:
URL = '/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/m/Music.htm'

f_music=open(URL, 'r')
soup_music = BeautifulSoup(f_music, 'lxml')

In [ ]:
paragraphs_m = soup_music.find_all('p')

links_m=[]
i=0

for p in paragraphs_m:
    i+=1
    all_links=p.find_all('a')
    for link in all_links:
        if 'href' in link.attrs:
            link_title=link.text
            link_paragraph=i
            links_m.append([link_title, link_paragraph])

link_positioning_df_m=pd.DataFrame(links_m, columns=['link', '#_paragraph']).drop_duplicates()
link_positioning_df_m['positioning']=round(link_positioning_df_m['#_paragraph']/len(paragraphs_m), 2)
link_positioning_df_m['link']=filter_article_name(link_positioning_df_m['link'])
link_positioning_df_m

In [ ]:
merged=link_positioning_df.merge(link_positioning_df_m, how='outer', left_on='link', right_on='link', )
columns_to_drop=['#_paragraph_x', '#_paragraph_y']
m=merged.drop(labels=columns_to_drop, axis=1)
m.reset_index(drop=True, inplace=True)
m

In [ ]:
# function for all html files

In [ ]:
for df in link_positioning_dfs:
    df.drop(labels='#_paragraph', axis=1)  

In [ ]:
def links_freq_vs_pos(dfs):
    merged_df=dfs[0]
    for df in dfs:
        merged_df=merged_df.merge(df, how='outer', left_on='link', right_on='link')
    return merged_df

links_freq_pos_dfs=links_freq_vs_pos(link_positioning_dfs)

In [ ]:
test = np.array([[1,2,3]]).flatten()
test

In [ ]:
#function to better take into account than the mean the positioning of the article

def find_positioning(array):
    bin_labels=['top', 'center', 'bottom']
    print(array)
    bins=[0, 0.33, 0.66, 1]
    cut_positioning=pd.cut(array, bins, labels=bin_labels)
    positioning_frequency=pd.value_counts(cut_positioning)
    print(positioning_frequency)
    most=positioning_frequency.apply('max')
    print(most)
    return positioning_frequency[positioning_frequency==most].index.values.astype(str)

find_positioning([0.1, 'NaN']) #example

In [ ]:
#mean_positioning=m.groupby(by=m['link'])
#mean_positioning_df=mean_positioning.groups
#dic=dict(mean_positioning_df)
#list_index=list(dic.values())
#type(list_index[0].values)
##m.loc[list_index,:]

In [ ]:
mean_positioning=m.groupby(by=m['link']).apply(lambda x: find_positioning(np.array(x.values).flatten()))
mean_positioning_df=pd.DataFrame(mean_positioning, columns=['mean_positioning'])
mean_positioning_df.reset_index(level=0, drop=True, inplace=True)
mean_positioning_df=mean_positioning_df.reset_index()
# mean_positioning_df.head()
# need to fix links showing up more than once

In [ ]:
mean_positioning_df

In [ ]:
links_freq_pos=links_freq_df.merge(mean_positioning_df, left_on='link', right_on='link')
links_freq_pos

In [ ]:
# function for all html files

links_freq_pos_dfs=links_freq_df.merge(mean_positioning_dfs, left_on='link', right_on='link')
links_freq_pos_dfs